In [ ]:
!pip install openai

In [ ]:
# @title Compare Images
from google.colab import userdata
isURL = False 
prompt = "What are in these images? Is there any difference between them?" 
images = ["https://upload.wikimedia.org/wikipedia/commons/9/9e/Ours_brun_parcanimalierpyrenees_1.jpg","https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Grosser_Panda.JPG/1200px-Grosser_Panda.JPG"] 

if isURL:
  print("URL option selected")
  from openai import OpenAI

  client = OpenAI(api_key = userdata.get('OPENAI_API_KEY'))
  messages = [
      {
        "role": "user",
        "content": [
          {"type": "text", "text": prompt},
        ],
      }
    ]
  for image in images:
    messages[0]["content"].append({
        "type": "image_url",
            "image_url": image
    })
  # print(messages)
  response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=messages,
    max_tokens=300,
  )
  print(response.choices[0].message.content)
else:
  print("Image File option selected")
  import base64
  import requests

  # OpenAI API Key
  api_key = userdata.get('OPENAI_API_KEY')

  # Function to encode the image
  def encode_image(image_path):
    with open(image_path, "rb") as image_file:
      return base64.b64encode(image_file.read()).decode('utf-8')
  messages = [{
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          }
        ]
      }]
  # Path to your image
  for image in images:
    image_path = image

    # Getting the base64 string
    base64_image = encode_image(image_path)
    messages[0]["content"].append(
        {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          })

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4-vision-preview",
    "messages": messages,
    "max_tokens": 300
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  print(response.json()["choices"][0]["message"]["content"])
  # print(response.json().choices[0].message.content)